# Notebook to plot the helicity angles of $a_{0}$

In [2]:
import ROOT
import numpy as np
import my_library.common_analysis_tools as ct
import my_library.constants as constants
import my_library.kinematic_cuts as kcuts
import pandas as pd

Welcome to JupyROOT 6.24/04


In [3]:
def get_gamma_cm(ey: float, s: float) -> float:
    # type: (float, float) -> float
    return (ey + 0.938272)/np.sqrt(s)
# def get_gamma_cm(ey, s):
#     return (ey + 0.938272)/np.sqrt(s)

def get_beta_cm(ey: float) -> float:
    return ey/(ey + 0.938272)


def get_beta(p: float, e: float) -> float:
    return p/e


def get_gamma(e: float, m: float) -> float:
    return e/m


In [4]:
def get_cm_vector(e: float, px: float, py: float, pz: float, ey: float, s: float):
    gamma = get_gamma_cm(ey, s)
    beta = get_beta_cm(ey)
    vec_cm_rotaton = np.array([[gamma, 0, 0, -1*beta*gamma],
                     [0, 1, 0, 0],
                     [0, 0, 1, 0],
                     [-1*beta*gamma, 0, 0, gamma]])
    vec_lab = np.array([e, px, py, pz]).reshape(4,1)
    return (vec_cm_rotaton @ vec_lab).reshape(1,4)


In [5]:
def get_rest_frame_vec(p4_decay_cm, p4_rest_cm, m_rest: float):
    """e in 0th element, px in 1st, py in 2nd, pz in 3rd"""
    p4_decay_cm   = p4_decay_cm.flatten()
    p4_rest_cm    = p4_rest_cm.flatten()
    beta = get_beta(np.sqrt(p4_rest_cm[1]*p4_rest_cm[1] + p4_rest_cm[2]*p4_rest_cm[2] + p4_rest_cm[3]*p4_rest_cm[3]), p4_rest_cm[0])
    gamma = get_gamma(p4_rest_cm[0], m_rest)
    beta_x = get_beta(p4_rest_cm[1], p4_rest_cm[0])
    beta_y = get_beta(p4_rest_cm[2], p4_rest_cm[0])
    beta_z = get_beta(p4_rest_cm[3], p4_rest_cm[0])

    p4_decay_cm = p4_decay_cm.reshape(-1, 1)

    rest_rotation = np.array([[gamma,         -gamma*beta_x,                             -gamma*beta_y,                             -gamma*beta_z                            ],
                              [-gamma*beta_x, 1 + (gamma - 1)*beta_x*beta_x/(beta*beta), (gamma - 1)*beta_x*beta_y/(beta*beta),     (gamma - 1)*beta_x*beta_z/(beta*beta)    ],
                              [-gamma*beta_y, (gamma - 1)*beta_y*beta_x/(beta*beta),     1 + (gamma - 1)*beta_y*beta_y/(beta*beta), (gamma - 1)*beta_y*beta_z/(beta*beta)    ],
                              [-gamma*beta_z, (gamma - 1)*beta_z*beta_x/(beta*beta),     (gamma - 1)*beta_z*beta_y/(beta*beta),     1 + (gamma - 1)*beta_z*beta_z/(beta*beta)]])

    return (rest_rotation @ p4_decay_cm).reshape(1, -1)

In [6]:
def get_y_hat(beam_cm_p4, m_cm_p4):
    beam_cm_p4 = beam_cm_p4.flatten()[1:]
    m_cm_p4 = m_cm_p4.flatten()[1:]
    cross_product = np.cross(beam_cm_p4, m_cm_p4)
    return cross_product/np.linalg.norm(cross_product)

def get_z_hat(m_cm_p4):
    p4_m_cm = m_cm_p4.flatten()[1:]
    return p4_m_cm/np.linalg.norm(p4_m_cm)

def get_x_hat(y_hat, z_hat):
    cross_product = np.cross(y_hat, z_hat)
    return cross_product/np.linalg.norm(cross_product)

def get_helicity_p3(decay_rest_p4, xhat, yhat, zhat):
    decay_rest_p4 = decay_rest_p4.flatten()[1:]
    return np.array([decay_rest_p4.dot(xhat), decay_rest_p4.dot(yhat), decay_rest_p4.dot(zhat)])

def cos_theta_helicty(p3_helicity):
    return p3_helicity[2]/np.linalg.norm(p3_helicity)

def phi_helicity(p3_helicity):
    return np.arctan2(p3_helicity[1], p3_helicity[0])

In [7]:
df_pipkmks = ct.get_dataframe("pipkmks", 'spring', 'data')

In [8]:
df_pipkmks = df_pipkmks.Define('kk_px', 'km_px + ks_px')
df_pipkmks = df_pipkmks.Define('kk_py', 'km_py + ks_py')
df_pipkmks = df_pipkmks.Define('kk_pz', 'km_pz + ks_pz')
df_pipkmks = df_pipkmks.Define('kk_E', 'km_E + ks_E')

In [9]:
df_pipkmks = df_pipkmks.Define("beam_p4", "ROOT::Math::PxPyPzEVector(e_beam, 0, 0, e_beam)")
df_pipkmks = df_pipkmks.Define("target_p4", "ROOT::Math::PxPyPzEVector(0, 0, 0, 0.938272)")

In [10]:
df_pipkmks = df_pipkmks.Define('cm_boost_vector', '(beam_p4 + target_p4).BoostToCM()')

In [11]:
df_pipkmks = df_pipkmks.Define('p_p4', 'ROOT::Math::PxPyPzEVector(p_px, p_py, p_pz, p_E)')
df_pipkmks = df_pipkmks.Define('pipkmks_p4', 'ROOT::Math::PxPyPzEVector(pipkmks_px, pipkmks_py, pipkmks_pz, pipkmks_E)')
df_pipkmks = df_pipkmks.Define('kk_p4', 'ROOT::Math::PxPyPzEVector(kk_px, kk_py, kk_pz, kk_E)')
df_pipkmks = df_pipkmks.Define('pip1_p4', 'ROOT::Math::PxPyPzEVector(pip1_px, pip1_py, pip1_pz, pip1_E)')


In [12]:
df_pipkmks = df_pipkmks.Define('cm_boost', "ROOT::Math::Boost(cm_boost_vector)")

In [13]:
df_pipkmks = df_pipkmks.Define('p_p4_cm', 'cm_boost(p_p4)')
df_pipkmks = df_pipkmks.Define('pipkmks_p4_cm', 'cm_boost(pipkmks_p4)')
df_pipkmks = df_pipkmks.Define('kk_p4_cm', 'cm_boost(kk_p4)')
df_pipkmks = df_pipkmks.Define('pip1_p4_cm', 'cm_boost(pip1_p4)')
df_pipkmks = df_pipkmks.Define('beam_p4_cm', 'cm_boost(beam_p4)')
df_pipkmks = df_pipkmks.Define('target_p4_cm', 'cm_boost(target_p4)')